In [1]:
import tkinter as tk
from tkinter import ttk
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH
import pandas as pd
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docxcompose.composer import Composer

def add_horizontal_line(doc, width_inches=2):
    # Add a paragraph for the line
    paragraph = doc.add_paragraph()
    
    # Add a table with one cell to hold the border line
    table = doc.add_table(rows=1, cols=1)
    cell = table.cell(0, 0)
    cell.width = Pt(width_inches * 72)  # Convert inches to points
    cell.text = ''  # No text in the cell

    # Access the cell's properties to set the border
    tcPr = cell._element.get_or_add_tcPr()
    tcBorders = OxmlElement('w:tcBorders')
    bottom = OxmlElement('w:bottom')
    bottom.set(qn('w:val'), 'single')
    bottom.set(qn('w:sz'), '8')  # Thickness of the line (in eighths of a point)
    bottom.set(qn('w:space'), '0')
    bottom.set(qn('w:color'), '000000')  # Black color

    # Append the bottom border to the table cell
    tcBorders.append(bottom)
    tcPr.append(tcBorders)

def add_bold_text(paragraph, text):
    run = paragraph.add_run(text)
    run.bold = True

def add_italic_text(paragraph, text):
    run = paragraph.add_run(text)
    run.italic = True

def format_client(input_text):
    # Split the text at every occurrence of "and" (case-insensitive)
    parts = input_text.split('and')
    
    # Combine parts with "and" followed by a newline after each part
    formatted_text = (' and\n'.join(part.strip() for part in parts)).strip()
    
    return formatted_text

def format_address(address):
    # Split the address into components based on known positions
    parts = address.split()
    
    # Assuming the address format is always correct and has a space between components
    street_address = ' '.join(parts[:-3])  # All but last three elements
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Format the address
    formatted_address = f"{street_address}\n{city} {state} {zip_code}"
    
    return formatted_address

def extract_address_components(address):
    # Split the address into parts based on spaces
    parts = address.split()
    
    # Ensure there are at least four parts to avoid errors
    if len(parts) < 4:
        raise ValueError("Address must contain at least a street address, city, state, and zip code.")
    
    # Extract the last three parts: city, state, and zip code
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Combine the remaining parts to form the street address
    street_name = ' '.join(parts[1:-3]).replace(',', '')
    
    # Return the street name, city, state, and zip code
    return street_name, city, state, zip_code

# Define containers for user input
client = input("Enter the client's name: ")
client_mailing_address = input("Enter the client's mailing address (Street Address, County/City, State Zip): ")
property_address = input("Enter the property address  (Street Address, County/City, State Zip): ")

street_name, city, state, zip_code = extract_address_components(property_address)

def get_zoning_info():
    # Load the Excel workbook
    workbook_path = 'Zoning_Workbook.xlsx'  # Ensure this is the correct file extension
    excel_file = pd.ExcelFile(workbook_path)

    # Display the tab names with corresponding numbers starting from 1
    tab_names = excel_file.sheet_names
    print("Select the county of the subject property:")
    for idx, name in enumerate(tab_names, start=1):
        print(f"{idx}: {name}")

    # Prompt user for tab number
    while True:
        try:
            tab_number = int(input("Enter the number corresponding to county of the subject property: ")) - 1
            if 0 <= tab_number < len(tab_names):
                break
            else:
                print("Invalid number. Please try again.")
        except ValueError:
            print("Please enter a valid number.")

    # Load the selected tab into a DataFrame
    selected_tab = tab_names[tab_number]
    df = pd.read_excel(workbook_path, sheet_name=selected_tab)

    # Display the first column values with corresponding numbers starting from 1
    first_col_values = df.iloc[:, 0].tolist()
    print("\nSelect the zoning of the subject property:")
    for idx, value in enumerate(first_col_values, start=1):
        print(f"{idx}: {value}")

    # Prompt user for value number from the first column
    while True:
        try:
            value_number = int(input("Enter the number corresponding to the zoning of the subject property: ")) - 1
            if 0 <= value_number < len(first_col_values):
                break
            else:
                print("Invalid number. Please try again.")
        except ValueError:
            print("Please enter a valid number.")

    # Get the selected county (the name of the selected tab)
    subject_county = selected_tab
    
    # Get the selected value from the first column
    selected_value = first_col_values[value_number]
    
    # Filter the DataFrame based on the selected value
    filtered_row = df[df.iloc[:, 0] == selected_value]

    # Extract values into specified containers
    if not filtered_row.empty:
        zone = filtered_row['Zone'].values[0] if 'Zone' in filtered_row.columns else None
        zone_abr = filtered_row['Zone Abbreviation'].values[0] if 'Zone Abbreviation' in filtered_row.columns else None
        ib = filtered_row['Issuing Body'].values[0] if 'Issuing Body' in filtered_row.columns else None
        zone_desc = filtered_row['Zone General Description'].values[0] if 'Zone General Description' in filtered_row.columns else None
    else:
        zone = zone_abr = ib = zone_desc = None

    # Return the extracted values in the correct order
    return subject_county, zone, zone_desc, zone_abr, ib

subject_county, zone, zone_desc, zone_abr, ib = get_zoning_info()

report_date = input("Enter the report date: ")
prg_ref_num = input("Enter the PRG reference number: ")
tax_account = input("Enter the tax account: ")

def lot_size_function():
    print("Please choose a unit of measurement for lot size:")
    print("1. Acres")
    print("2. Square Feet")
    choice = input("Enter the number of your choice (1 or 2): ").strip()
    if choice == "1":
        lot_unit = "acres"
        lot_size_input = input("Enter the number of acres: ").strip()
    elif choice == "2":
        lot_unit = "square feet"
        lot_size_input = input("Enter the number of lot square feet: ").strip()
    else:
        raise ValueError("Invalid choice. Please enter 1 for Acres or 2 for Square Feet.")
    try:
        lot_size = int(lot_size_input)
        lot_size_formatted = f"{lot_size:,}"
    except ValueError:
        raise ValueError("Invalid input for lot size. Please enter a valid integer.")
    return lot_size, lot_size_formatted, lot_unit  # Return the values

lot_size, lot_size_formatted, lot_unit = lot_size_function()

def parcel_shape_function():
    print("What is the shape of the parcel?")
    print("1. Rectangular")
    print("2. Irregular")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        parcel_shape = "rectangular"
    elif choice == "2":
        parcel_shape = "irregular"
    else:
        print("Invalid choice. Please enter 1 for Rectangular or 2 for Irregular.")
    return parcel_shape

parcel_shape = parcel_shape_function()

def vacant_or_improved():
    print("Is the subject property vacant or improved? ")
    print("1. Vacant")
    print("2. Improved")
    choice = input("Enter the number of your choice: ").strip()
    
    vac_or_imp = None
    building_sf = None
    building_sf_formatted = None
    year_built = None
    stories = None
    bedrooms = None
    full_bath = None
    half_bath = None
    property_type = None
    
    if choice == "1":
        vac_or_imp = "vacant"
    elif choice == "2":
        vac_or_imp = "improved"
        building_sf = input("Please enter the building square feet: ").strip()
        
        try:
            # Convert to an integer for formatting
            building_sf_int = int(building_sf)
            building_sf_formatted = f"{building_sf_int:,}"
        except ValueError:
            print("Invalid input for building square feet. Please enter a number.")
            return None  # Early exit on error
        
        year_built = input("Enter the year built: ").strip()
        print("Please choose a building type:")
        print("1. Residence")
        print("2. Flex")
        print("3. Medical Office")
        print("4. Mixed-Use")
        print("5. Farm")
        building_type_choice = input("Enter the number corresponding to your choice: ").strip()
        
        if building_type_choice == "1":
            property_type = "Residence"
            stories = input("Enter the number of stories: ").strip()
            bedrooms = input("Enter the number of bedrooms: ").strip()
            full_bath = input("Enter the number of full bathrooms: ").strip()
            half_bath = input("Enter the number of half bathrooms: ").strip()
        elif building_type_choice == "2":
            property_type = "Flex"
        elif building_type_choice == "3":
            property_type = "Medical Office"
        elif building_type_choice == "4":
            property_type = "Mixed-Use"
            stories = input("Enter the number of stories: ").strip()
        elif building_type_choice == "5":
            property_type = "Farm"
            outbuilding_condition = input("Enter the condition of the outbuildings: ").strip()
        else:
            print("Invalid choice. Please enter the number corresponding to your choice.")
    
    return vac_or_imp, building_sf, building_sf_formatted, year_built, stories, bedrooms, full_bath, half_bath, property_type, outbuilding_condition

vac_or_imp, building_sf, building_sf_formatted, year_built, stories, bedrooms, full_bath, half_bath, property_type, outbuilding_condition = vacant_or_improved()




if vac_or_imp == "improved":
    if lot_unit == "acres":
        far = round((int(building_sf)/(int(lot_size)*43560)),4)
    else:
        far = round((int(building_sf)/int(lot_size)),4)

grantor = input("Enter the grantor of the last transfer: ")
grantee = input("Enter the grantee of the last transfer: ")
transfer_date = input("Enter the date of the last transfer: ")
liber = input("Enter the deed Liber: ")
folio = input("Enter the deed Folio: ")

def transfer_terms():
    print("What were the terms of this transfer?")
    print("1. Arms-Length")
    print("2. Non-Arms-Length")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        terms = "arms-length"
    elif choice == "2":
        terms = "non-arms-length"
    else:
        print("Invalid choice. Please enter the number corresponding to your choice.")
    return terms

terms = transfer_terms()
map_num = input("Enter the map number: ")
grid = input("Enter the grid number: ")
parcel = input("Enter the parcel number: ")
election_district = input("Enter the election district: ")
census_tract = input("Enter the census tract: ")
road_frontage = input("Enter the total road frontage of the subject property (feet): ")

def choose_value_type():
    global valuation_choice, decedent, date_of_valuation, date_of_inspection, estate_number, estate

    # Display the options to the user
    print("Please choose the type of valuation:")
    print("1. Current Market Value 'As-Is'")
    print("2. Retrospective Market Value")

    # Loop until a valid choice is made
    while True:
        choice = input("Enter the number of your choice (1 or 2): ").strip()
        if choice == "1":
            # Set the global variables
            valuation_choice = "Current Market Value 'As-Is'"
            decedent = None
            date_of_valuation = input("Enter the date of valuation (e.g., MM/DD/YYYY): ").strip()
            date_of_inspection = None
            estate_number = None
            estate = None
            break
        elif choice == "2":
            print("Is this appraisal for an estate?")
            print("1. Yes")
            print("2. No")
            while True:
                estate_choice = input("Enter the number of your choice (1 or 2): ").strip()
                if estate_choice == "1":
                    estate = 'yes'
                    valuation_choice = "Retrospective Market Value"
                    decedent = input("Please enter the name of the decedent: ").strip()
                    date_of_valuation = input("Please enter the date of death (e.g., MM/DD/YYYY): ").strip()
                    estate_number = input("Please enter the estate number: ").strip()
                    date_of_inspection = input("Please enter the date of inspection (e.g., MM/DD/YYYY): ").strip()
                    break
                elif estate_choice == "2":
                    estate = 'no'
                    valuation_choice = "Retrospective Market Value"
                    decedent = None
                    date_of_valuation = input("Enter the date of valuation (e.g., MM/DD/YYYY): ").strip()
                    estate_number = None
                    date_of_inspection = input("Please enter the date of inspection (e.g., MM/DD/YYYY): ").strip()
                    break
                else:
                    print("Invalid choice. Please enter 1 for Yes or 2 for No.")
            break
        else:
            print("Invalid choice. Please enter 1 for Current Market Value 'As-Is' or 2 for Retrospective Market Value.")

# Call the function to populate the containers
choose_value_type()

def interest_appraised_function():
    print("What is the interest appraised?")
    print("1. Fee Simple.")
    print("2. Leased Fee.")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        interest_appraised = "Fee Simple"
    elif choice == "2":
        interest_appraised = "Leased Fee"
    else:
        print("Invalid choice. Please enter the number corresponding to your choice.")
    return interest_appraised

interest_appraised = interest_appraised_function()

prepared_by = input("Prepared by: ")

def set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000"):
    p = paragraph._element
    pPr = p.find(qn('w:pPr'))
    if pPr is None:
        pPr = OxmlElement('w:pPr')
        p.append(pPr)

    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), fill_color)
    pPr.append(shd)

    for border_type in ['top', 'bottom', 'left', 'right']:
        border = OxmlElement(f'w:{border_type}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '4')
        border.set(qn('w:space'), '0')
        border.set(qn('w:color'), border_color)
        pPr.append(border)

def set_cell_paragraph_formatting(paragraph):
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_before = Pt(6)

def set_column_width(cell, width):
    cell.width = Inches(width)
    tcPr = cell._element.get_or_add_tcPr()
    tcW = OxmlElement('w:tcW')
    tcW.set(qn('w:w'), str(int(width * 1440)))  # width in twips (1 inch = 1440 twips)
    tcW.set(qn('w:type'), 'dxa')
    tcPr.append(tcW)

if vac_or_imp == "vacant":
    property_paragraph = ""
elif property_type == "Residence" and half_bath not in ["zero",0,"0","Zero"]:
    property_paragraph = f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms, {full_bath} full bathrooms and {half_bath} half bathrooms. The building has {building_sf_formatted} square feet of gross living area and was constructed in {year_built} (according to the {subject_county} land records).""" 
elif property_type == "Residence" and half_bath in ["zero",0,"0","Zero"]:
    property_paragraph = f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms and {full_bath} bathrooms. The building has {building_sf_formatted} square feet of gross living area and was constsubject_cructed in {year_built} (according to the {ounty} land records.)"""
elif property_type == "Flex":
    property_paragraph = f"""This property is improved with a flex building that has approximately {building_sf_formatted} square feet of gross building area, which results in a floor-to-area (FAR) ratio of {far}. This building was constructed in {year_built}."""
elif property_type == "Medical Office":
    property_paragraph = f"""This property is improved with an office building that is currently used for medical office space. The office building has {building_sf_formatted} square feet of gross building area and was constructed in {year_built}."""
elif property_type == "Mixed-Use":
    property_paragraph = f"""At the time of the property inspection, the subject property is improved with a {stories}-story mixed-use building that has approximately {building_sf_formatted} square feet of gross building area."""
elif property_type == "Farm":
    property_paragraph = f"""This property is improved with multiple farm buildings: . Details of the subject property buildings are shown in the table below. Overall, the buildings were in {outbuilding_condition} condition."""
else:
    property_paragraph = ""

# Load the base template
doc = Document('Base Template.docx')



# Set the desired margins (in inches)
left_margin = Inches(1)
right_margin = Inches(1)

# Apply margins to all sections in the document
for section in doc.sections:
    section.left_margin = left_margin
    section.right_margin = right_margin

# Add space before title paragraph
doc.add_paragraph()  # Adds a blank paragraph for spacing








##### Title Page








# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")

# Set font properties
title_run.bold = True
title_run.font.size = Pt(36)
title_run.font.name = 'Times New Roman'

# Set paragraph alignment to center
title_paragraph.alignment = 1  # Center alignment

doc.add_paragraph() 

# Add a table with 9 rows and 2 columns
table = doc.add_table(rows=4, cols=2)

# Set the table to not autofit
table.autofit = False

# Set the preferred width for the entire table
# table.width = Inches(4.75)  # Adjust this value as needed

# Set the width for each column
column_widths = [Inches(2.44), Inches(2.19)]  # Adjust these values as needed
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

# Center align the table in the document
table.alignment = WD_ALIGN_PARAGRAPH.CENTER

exposure_time = "Exposure Time"
exposure_time_def = "The estimated length of time the property interest being appraised would have been offered on the market prior to the hypothetical consummation of a sale at market value on the effective date of the appraisal; a retrospective estimate based on an analysis of past events assuming a competitive and open market."
marketing_time = "Marketing Time"
marketing_time_def = "The time it takes an interest in real property to sell on the market subsequent to the date of appraisal."

left_col_vals = [
    "CLIENT",
    "PROPERTY ADDRESS",
    "REPORT DATE",
    "PRG REFERENCE NUMBER"
]

right_col_vals = [
    f"{client}",
    f"{format_address(property_address)}",
    f"{report_date}",
    f"{prg_ref_num}"
]

for i in range(4):
    # Left column with values made bold
    cell_left = table.cell(i, 0)
    paragraph_left = cell_left.paragraphs[0]
    add_bold_text(paragraph_left, left_col_vals[i])

    # Right column with values from the provided list
    cell_right = table.cell(i, 1)
    paragraph_right = cell_right.paragraphs[0]
    paragraph_right.add_run(right_col_vals[i])

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        for run in paragraph.runs:
            run.font.name = 'Times New Roman'
            run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(6)
        paragraph.paragraph_format.space_after = Pt(0)


doc.add_paragraph()  
doc.add_paragraph()  

# Create a new paragraph for the address
address_paragraph = doc.add_paragraph()
address_paragraph.alignment = 1  # Center alignment

# Add address text to the paragraph
address_run = address_paragraph.add_run(property_address)

# Set font properties for the address
address_run.bold = True
address_run.italic = True
address_run.font.size = Pt(12)
address_run.font.name = 'Times New Roman'

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image
doc.add_paragraph()

# Add the PRG Logo image to the same paragraph
logo_paragraph = doc.add_paragraph()
logo_paragraph.alignment = 1  # Center alignment

# Add the PRG Logo image to the paragraph
logo_paragraph.add_run().add_picture('PRG Logo.jpg', width=Inches(2.0))  # Adjust width as needed

# Add a page break
doc.add_page_break()






#### Page 2







doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")
title_run.bold = True
title_run.font.size = Pt(14)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("ADDRESS")
title_run.bold = True
title_run.font.size = Pt(12)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Creates "Address" section
paragraph = doc.add_paragraph(format_address(property_address))
paragraph.alignment = 1
paragraph.paragraph_format.space_before = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
legal_desc_paragraph = doc.add_paragraph()
legal_desc_run = legal_desc_paragraph.add_run("LEGAL DESCRIPTION")
legal_desc_run.bold = True
legal_desc_run.font.size = Pt(12)
legal_desc_run.font.name = 'Times New Roman'
legal_desc_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Creates legal description
par1 = doc.add_paragraph(f"Tax Account: {tax_account}")
par1.alignment = 1
par1.paragraph_format.space_before = Pt(6)
par2 = doc.add_paragraph(f"{lot_size} {lot_unit} of land")
par2.alignment = 1
par3 = doc.add_paragraph(f"Liber {liber}/Folio {folio}")
par3.alignment = 1
par4 = doc.add_paragraph(f"Map {map_num}/Grid {grid}/Parcel {parcel}")
par4.alignment = 1
par5 = doc.add_paragraph(f"{election_district} Election District")
par5.alignment = 1
par6 = doc.add_paragraph(f"Census Tract: {census_tract}")
par6.alignment = 1
par7 = doc.add_paragraph(f"{city} {state} {zip_code}")
par7.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("AS OF")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Creates the "As Of" section
par8 = doc.add_paragraph(f"{date_of_valuation}")
par8.alignment = 1
par8.paragraph_format.space_before = Pt(6)
par9 = doc.add_paragraph(f"{valuation_choice}")
par9.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("FOR")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Creates "For" section
par10 = doc.add_paragraph(f"{client}")
par10.alignment = 1
par10.paragraph_format.space_before = Pt(6)
par11 = doc.add_paragraph(f"{format_address(client_mailing_address)}")
par11.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("PREPARED BY")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Creates the "Prepared By" section
par12 = doc.add_paragraph(f"{prepared_by}")
par12.alignment = 1
par12.paragraph_format.space_before = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Copyright © 2024")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Pugh Real Estate Group, LLC")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Real Estate Appraisers")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("P.O. Box 378")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Frederick, Maryland 21705")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a page break
doc.add_page_break()







#### Page 3







# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()

# "PUGH"
prep_run_p = prep_paragraph.add_run("P")
prep_run_p.font.size = Pt(18)
prep_run_p.font.name = 'Copperplate Gothic Bold'

prep_run_ugh = prep_paragraph.add_run("UGH")
prep_run_ugh.font.size = Pt(16)
prep_run_ugh.font.name = 'Copperplate Gothic Bold'

# "REAL"
prep_run_r = prep_paragraph.add_run(" R")
prep_run_r.font.size = Pt(18)
prep_run_r.font.name = 'Copperplate Gothic Bold'

prep_run_eal = prep_paragraph.add_run("EAL")
prep_run_eal.font.size = Pt(16)
prep_run_eal.font.name = 'Copperplate Gothic Bold'

# "ESTATE"
prep_run_r = prep_paragraph.add_run(" E")
prep_run_r.font.size = Pt(18)
prep_run_r.font.name = 'Copperplate Gothic Bold'

prep_run_estate = prep_paragraph.add_run("STATE")
prep_run_estate.font.size = Pt(16)
prep_run_estate.font.name = 'Copperplate Gothic Bold'

# "GROUP"
prep_run_g = prep_paragraph.add_run(" G")
prep_run_g.font.size = Pt(18)
prep_run_g.font.name = 'Copperplate Gothic Bold'

prep_run_roup = prep_paragraph.add_run("ROUP,")
prep_run_roup.font.size = Pt(16)
prep_run_roup.font.name = 'Copperplate Gothic Bold'

# LLC
# "GROUP"
prep_run_g = prep_paragraph.add_run(" LLC")
prep_run_g.font.size = Pt(18)
prep_run_g.font.name = 'Copperplate Gothic Bold'

# Align the paragraph to center
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("REAL ESTATE APPRAISERS")
prep_run.bold = False
prep_run.font.size = Pt(10)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("POST OFFICE BOX 378")
prep_run.bold = False
prep_run.font.size = Pt(6)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("FREDERICK, MARYLAND 21705")
prep_run.bold = False
prep_run.font.size = Pt(6)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("(301) 898-1178 ")
prep_run.bold = False
prep_run.font.size = Pt(8)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_run.font.underline = True 
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{report_date}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{client}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{client_mailing_address}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Re:     {street_name}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"           {city} {state} {zip_code}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Dear {client},")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

def add_paragraph_based_on_choice():
    global valuation_choice, decedent, date_of_valuation, report_date

    # Add the title paragraph with centered text based on the valuation choice
    if valuation_choice == "Current Market Value 'As-Is'":
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""You asked me to provide you with an appraisal value for the property identified above. This appraisal report provides an opinion of value as of {report_date}, which was the date of inspection. The attached report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


    elif valuation_choice == "Retrospective Market Value" and estate == 'yes':
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""Recently, you asked me to provide you with a retrospective market value for the property identified above. This appraisal report provides an opinion of value as of {date_of_valuation}, which is the date of death of {decedent}. The attached appraisal report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)
        
    else:
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""Recently, you asked me to provide you with a retrospective market value for the property identified above. This appraisal report provides an opinion of value as of {date_of_valuation}. The attached appraisal report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


add_paragraph_based_on_choice()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("SUMMARY OF SITE, ZONING, AND IMPROVEMENTS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"""The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}.""")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

if vac_or_imp != 'vacant':
    prep_paragraph = doc.add_paragraph()
    prep_run = prep_paragraph.add_run(f"{property_paragraph}")
    prep_run.bold = False
    prep_run.font.size = Pt(12)
    prep_run.font.name = 'Times New Roman'
    prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
    prep_paragraph.paragraph_format.space_before = Pt(6)
    prep_paragraph.paragraph_format.space_after = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)


# Add a page break
doc.add_page_break()








#### Page 4









# First paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("EXTRAORDINARY ASSUMPTIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_after = Pt(6)

# Second paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Uniform Standard of Professional Appraisal Practice (USPAP) defines an Extraordinary Assumption as, “an assumption, directly related to a specific assignment, as of the effective date of the assignment result, which, if found to be false, could alter the appraiser’s opinions or conclusions.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Third paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No extraordinary assumptions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Fourth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("HYPOTHETICAL CONDITIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Fifth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("USPAP defines a Hypothetical Condition as, “that which is contrary to what is known the appraiser to exist on the effective date of the assignment results but is used for the purpose of analysis.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Sixth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No hypothetical conditions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("After careful consideration of all the factors which create value, it is my opinion that the subject property has a value as follows:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing

def set_cell_borders(cell, top_color="000000", bottom_color="000000", left_color="000000", right_color="000000", border_width=4):
    """Set the borders for a cell with individual colors for top, bottom, left, and right borders."""
    tc_pr = cell._element.xpath('.//w:tcPr')
    if not tc_pr:
        tc_pr = OxmlElement('w:tcPr')
        cell._element.insert(0, tc_pr)
    else:
        tc_pr = tc_pr[0]

    borders = OxmlElement('w:tcBorders')
    border_attrs = {
        'w:top': top_color,
        'w:left': left_color,
        'w:bottom': bottom_color,
        'w:right': right_color,
        'w:insideH': 'FFFFFF',  # inside borders color, typically set to white
        'w:insideV': 'FFFFFF'   # inside borders color, typically set to white
    }

    for border, color in border_attrs.items():
        border_elem = OxmlElement(border)
        border_elem.set(qn('w:val'), 'single')
        border_elem.set(qn('w:sz'), str(border_width))  # Border width (1/8 pt)
        border_elem.set(qn('w:space'), '0')
        border_elem.set(qn('w:color'), color)  # Set border color
        borders.append(border_elem)

    tc_pr.append(borders)

def set_cell_fill(cell, color):
    """Set the fill color for a cell."""
    shading = OxmlElement('w:shd')
    shading.set(qn('w:fill'), color)
    cell._element.get_or_add_tcPr().append(shading)

# Add a table with 2 rows and 4 columns
table = doc.add_table(rows=2, cols=4)

# Define the headers for the first row
headers_1 = ["APPRAISAL PREMISE", "INTEREST APPRAISED", "DATE OF VALUE", "VALUE CONCLUSION"]
headers_2 = [f"{valuation_choice}", f"{interest_appraised}", f"{date_of_valuation}", "$"]

# Populate the header row with formatting
for col, header in enumerate(headers_1):
    cell = table.cell(0, col)
    cell.text = header
    set_cell_borders(cell, top_color="FFFFFF", bottom_color="000000", left_color="FFFFFF", right_color="FFFFFF", border_width=4)
    # Set text to bold
    for paragraph in cell.paragraphs:
        run = paragraph.runs[0]
        run.bold = True
        run.font.name = "Times New Roman"
        run.font.size = Pt(12)
        paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Center align the header

# Populate the second row with data from headers_2
for col, header in enumerate(headers_2):
    cell = table.cell(1, col)  # Use row 1 (second row)
    cell.text = header
    if col == 0:
        # For the first cell of the second row, set the right border color to "FFFFFF"
        set_cell_borders(cell, top_color="000000", bottom_color="FFFFFF", left_color="FFFFFF", right_color="000000", border_width=4)
    else:
        set_cell_borders(cell, top_color="000000", bottom_color="FFFFFF", left_color="FFFFFF", right_color="BFBFBF", border_width=4)
        set_cell_fill(cell, color="BFBFBF")
    
    # Set text formatting
    for paragraph in cell.paragraphs:
        if col == 0:
            run = paragraph.runs[0]
            run.font.name = "Times New Roman"
            run.font.size = Pt(12)
            run.font.italic = True
            paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
        else:
            run = paragraph.runs[0]
            run.font.name = "Times New Roman"
            run.font.size = Pt(12)
            run.font.italic = False
            paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Center align the content
    

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("If you have any questions concerning this appraisal or the value conclusions developed in this report, please do not hesitate to contact me.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Sincerely,")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Add a page break
doc.add_page_break()





#### Page 5







paragraph = doc.add_paragraph()
run = paragraph.add_run(" TABLE OF CONTENTS                                                                                 Page")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
paragraph.paragraph_format.space_after = Pt(3)
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")  

# Add a table with 21 rows and 2 columns
table = doc.add_table(rows=21, cols=2)

# Define the values for each column
left_column_values = [
    "SUMMARY OF IMPORTANT FACTS AND CONCLUSIONS",
    "CERTIFICATE OF APPRAISAL",
    "SCOPE OF THE APPRAISAL",
    "ASSUMPTIONS AND LIMITING CONDITIONS",
    "KEY TERMS FOR THIS APPRAISAL REPORT",
    "IDENTIFICATION OF THE PROPERTY",
    "SUBJECT PROPERTY HISTORY",
    "REGIONAL DATA",
    "NEIGHBORHOOD DATA",
    "REAL ESTATE TAX ANALYSIS",
    "MARKET ANALYSIS",
    "ZONING",
    "SITE DATA",
    "DESCRIPTION OF THE IMPROVEMENTS",
    "HIGHEST AND BEST USE",
    "VALUATION METHODOLOGY",
    "SITE VALUATION",
    "COST APPROACH",
    "INCOME APPROACH",
    "SALES COMPARISON APPROACH",
    "RECONCILIATION OF VALUE INDICATIONS"
]

right_column_values = [
    "5", "6", "7", "8", "10", "12", "14", "15", "25", "29",
    "31", "38", "40", "47", "57", "61", "62", "63", "66", "75", "84"
]

# Populate the table
for row_index in range(21):
    # Set values in the first column (left side)
    cell_left = table.cell(row_index, 0)
    cell_left.text = left_column_values[row_index]
    
    # Set values in the second column (right side)
    cell_right = table.cell(row_index, 1)
    cell_right.text = right_column_values[row_index]
    
    # Set the font style for both cells and adjust paragraph spacing
    for cell in [cell_left, cell_right]:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.name = 'Times New Roman'
                run.font.size = Pt(12)
                
                # Fix for setting the font name explicitly
                r = run._element
                rPr = r.find(qn('w:rPr'))
                rFonts = OxmlElement('w:rFonts')
                rFonts.set(qn('w:ascii'), 'Times New Roman')
                rFonts.set(qn('w:hAnsi'), 'Times New Roman')
                rPr.append(rFonts)

            # Set paragraph spacing
            paragraph_format = paragraph.paragraph_format
            paragraph_format.space_before = Pt(6)  # Space before the paragraph
            paragraph_format.space_after = Pt(0)  # Space after the paragraph

            # Center text in the right column
            if cell == cell_right:
                paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Adjust the column widths
for col_idx in range(2):
    for row in table.rows:
        cell = row.cells[col_idx]
        if col_idx == 0:
            cell.width = Inches(6.25)  # Adjust as needed for the left column width
        else:
            cell.width = Inches(0.25)  # Adjust as needed for the right column width
            
paragraph = doc.add_paragraph()
prep_run = paragraph.add_run("ADDENDUM:")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
paragraph.paragraph_format.space_before = Pt(18)
paragraph.paragraph_format.space_after = Pt(0)

paragraph = doc.add_paragraph()
run = paragraph.add_run("APPRAISER QUALIFICATIONS")
run.bold = False
run.font.name = 'Times New Roman'
run.font.size = Pt(12)
paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
paragraph.paragraph_format.space_before = Pt(0)
paragraph.paragraph_format.space_after = Pt(0)

paragraph = doc.add_paragraph()
run = paragraph.add_run("ENGAGEMENT LETTER")
run.bold = False
run.font.name = 'Times New Roman'
run.font.size = Pt(12)
paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
paragraph.paragraph_format.space_before = Pt(0)
paragraph.paragraph_format.space_after = Pt(0)
            
# Add a page break
doc.add_page_break()






#### Page 6









paragraph = doc.add_paragraph()
run = paragraph.add_run("SUMMARY OF IMPORTANT FACTS AND CONCLUSIONS")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Define the table data for the first column
left_column_values1 = [
    "Address", "Date of Value Estimate", "Date of Appraisal Report",
    "Property Rights Appraised", "Land Area", "Improvements", 
    "Zoning", "Intended Use", "Client/Intended User"
]

# Define the table data for the right column
right_column_values = [
    f"{property_address}", 
    f"{date_of_valuation}", 
    f"{report_date}",
    f"{interest_appraised}.",
    f"{lot_size} {lot_unit}.", 
    f"{property_paragraph}", 
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}.", 
    "", 
    f"{client}"
]

# Add a 9x2 table beneath the previous content
table = doc.add_table(rows=9, cols=2)

# Adjust column widths
left_column_width = 1.5  # Width for the left column in inches
right_column_width = 5  # Width for the right column in inches

# Loop through each row to add the left column and right column text with formatting
for idx in range(len(left_column_values1)):
    # Add value to the first column
    left_cell = table.cell(idx, 0)
    left_paragraph = left_cell.paragraphs[0]
    run = left_paragraph.add_run(left_column_values1[idx])
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)
    set_cell_paragraph_formatting(left_paragraph)
    
    # Set width for the left column
    set_column_width(left_cell, left_column_width)
    
    # Add value to the second column
    right_cell = table.cell(idx, 1)
    right_paragraph = right_cell.paragraphs[0]
    right_run = right_paragraph.add_run(right_column_values[idx])
    right_run.font.name = 'Times New Roman'
    right_run.font.size = Pt(12)
    set_cell_paragraph_formatting(right_paragraph)
    
    # Set width for the second column
    set_column_width(right_cell, right_column_width)
    
paragraph = doc.add_paragraph()
run = paragraph.add_run("HIGHEST AND BEST USE")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
paragraph.paragraph_format.space_before = Pt(6)
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

left_column_values2 = ["As Vacant", "As Improved"]
right_column_values2 = ["", ""]

table = doc.add_table(rows=2, cols=2)
    
for idx in range(len(left_column_values2)):
    # Add value to the first column
    left_cell = table.cell(idx, 0)
    left_paragraph = left_cell.paragraphs[0]
    run = left_paragraph.add_run(left_column_values2[idx])
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)
    set_cell_paragraph_formatting(left_paragraph)
    
    # Set width for the left column
    set_column_width(left_cell, left_column_width)

for idx in range(len(right_column_values2)):
    # Add value to the first column
    right_cell = table.cell(idx, 1)
    right_paragraph = right_cell.paragraphs[0]
    run = right_paragraph.add_run(right_column_values2[idx])
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)
    set_cell_paragraph_formatting(right_paragraph)
    
    # Set width for the left column
    set_column_width(right_cell, right_column_width)
    
paragraph = doc.add_paragraph()
run = paragraph.add_run("VALUATION")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
paragraph.paragraph_format.space_before = Pt(6)
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Add a table with 5 rows and 3 columns
table = doc.add_table(rows=5, cols=3)

# Set the width for each column (in inches)
column_widths = [Inches(2), Inches(2.5), Inches(1.5)]  # Adjust these values as needed

# Adjust column widths
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

# Merge all cells in the first column
cell_to_merge = table.cell(0, 0)
for row in range(1, 5):
    cell_to_merge = cell_to_merge.merge(table.cell(row, 0))

# Set the text for the merged cell
cell_to_merge.text = f'{valuation_choice} as of {date_of_valuation}'

# Set the specified text for the second column
second_column_texts = [
    "Site Valuation",
    "Cost Approach",
    "Income Capitalization Approach",
    "Sales Comparison Approach",
    "Conclusion"
]

# Set the specified text for the third column
third_column_texts = [
    "-",
    "-",
    "-",
    "-",
    "-"
]

for row in range(5):
    table.cell(row, 1).text = second_column_texts[row]
    table.cell(row, 2).text = third_column_texts[row]

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        run = paragraph.runs[0]  # Get the first run (the text portion)
        
        # Set font to Times New Roman and size to 12
        run.font.name = 'Times New Roman'
        run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_after = Pt(0)  # Adjust this value as needed

# Add a page break
doc.add_page_break()







#### Page 7







paragraph = doc.add_paragraph()
run = paragraph.add_run("CERTIFICATE OF APPRAISAL")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("I certify that, to the best of my knowledge and belief: ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0)

# Add a table with 5 rows and 3 columns
table = doc.add_table(rows=13, cols=2)

# Set the width for each column (in inches)
column_widths = [Inches(0.25), Inches(6)]  # Adjust these values as needed

# Adjust column widths
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

texts = [
    "The statements of fact contained in this report are true and correct.",
    "The reported analyses, opinions, and conclusions are limited only by the reported assumptions and limiting conditions and are my personal, impartial, and unbiased professional analyses, opinions, and conclusions.",
    "I have no present or prospective interest in the property that is the subject of this report and no personal interest with respect to the parties involved.",
    "I have not appraised this property within the last three years.",
    "I have no bias with respect to the property that is the subject of this report or to the parties involved with this assignment.",
    "My engagement in this assignment was not contingent upon developing or reporting predetermined results.",
    "My compensation for completing this assignment is not contingent upon the development or reporting of a predetermined value or direction in value that favors the cause of the client, the amount of the value opinion, the attainment of a stipulated result, or the occurrence of a subsequent event directly related to the intended use of this appraisal.",
    "My analyses, opinions, and conclusions were developed, and this report has been prepared, in conformity with the Uniform Standards of Professional Appraisal Practice.",
    "I have made a personal inspection of the property that is the subject of this report.",
    "No one provided significant real property appraisal assistance to the person signing this certification.",
    "The reported analyses, opinions, and conclusions were developed, and this report has been prepared, in conformity with the Code of Professional Ethics and Standards of Professional Appraisal Practice of the Appraisal Institute.",
    "The use of this report is subject to the requirements of the Appraisal Institute relating to review by its duly authorized representatives.",
    "As of the date of this report, I have completed the Standards and Ethics Education Requirement of the Appraisal Institute for Designated Members."
]

# Add data to the table
for i in range(13):
    # Left column with numbers formatted as "1)", "2)", etc.
    cell_left = table.cell(i, 0)
    cell_left.text = f"{i+1})"
    
    # Right column with values from the provided list
    cell_right = table.cell(i, 1)
    cell_right.text = texts[i]

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        run = paragraph.runs[0]  # Get the first run (the text portion)
        
        # Set font to Times New Roman and size to 12
        run.font.name = 'Times New Roman'
        run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(6)
        paragraph.paragraph_format.space_after = Pt(0)  # Adjust this value as needed

# Add a page break
doc.add_page_break()



        




#### Page 7








paragraph = doc.add_paragraph()
run = paragraph.add_run("SCOPE OF THE APPRAISAL")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")        
     
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Real estate is valued based using the following techniques – Site Valuation, Cost, Income and Sales Comparison Approaches – which are defined below:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0)   

# Add a table with 4 rows and 2 columns
table = doc.add_table(rows=4, cols=2)

# Set the width for each column (in inches)
column_widths = [Inches(1.5), Inches(5)]  # Adjust these values as needed

# Adjust column widths
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

left_col_vals = [
    "SITE VALUATION",
    "COST",
    "INCOME CAPITALIZATION APPROACH",
    "SALES COMPARISON APPROACH"
]

right_col_vals = [
    "This approach to value is the estimation of value of a subject site, as vacant and ready to be improved, based upon factual data extracted from sales of similar or reasonably similar properties.",
    "This approach to value is utilized to reflect the value of the property by adding together the estimated value of the land, as vacant, and the reproduction cost new of the improvements, less the accrued depreciation.",
    "This approach estimates value by the capitalization of the net income stream.",
    "This approach to value is the estimation of value of a subject property based upon factual data extracted from sales of similar or reasonably similar properties."
]

# Add data to the table
for i in range(4):
    # Left column with values made bold
    cell_left = table.cell(i, 0)
    cell_left.text = left_col_vals[i]
    
    # Make the left cell text bold
    for paragraph in cell_left.paragraphs:
        for run in paragraph.runs:
            run.bold = True  # Set the text to bold

    # Right column with values from the provided list
    cell_right = table.cell(i, 1)
    cell_right.text = right_col_vals[i]

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        run = paragraph.runs[0]  # Get the first run (the text portion)
        
        # Set font to Times New Roman and size to 12
        run.font.name = 'Times New Roman'
        run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(6)
        paragraph.paragraph_format.space_after = Pt(0)  # Adjust this value as needed

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("These data will be gathered from the subject area and all pertinent data gathered will be presented, discussed, and considered.  Confirmation of data will be made as needed to assure an understanding of the actions of the market.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0)   

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("The appraiser has performed the following in arriving at a final value estimate for the subject property under market value “as-is.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)   

def add_paragraph_based_on_inspector():
    global prepared_by, valuation_choice, date_of_valuation, date_of_inspection

    # Create the text based on the valuation choice
    if valuation_choice == "Current Market Value 'As-Is'":
        text = (
            f"{prepared_by} personally inspected the subject property on {date_of_valuation}. "
            "The scope of this inspection included the interior and exterior of the buildings, site, and surrounding area."
        )
    else:
        text = (
            f"{prepared_by} personally inspected the subject property on {date_of_inspection}. "
            "The scope of this inspection included the interior and exterior of the buildings, site, and surrounding area."
        )
    
    return text

# Get the text from the function
inspector_paragraph_text = add_paragraph_based_on_inspector()

# List of items for the bulleted list, starting with the inspector paragraph
items = [
    inspector_paragraph_text, 
    'Gathered and analyzed data on comparable improved sales in the market area from the Maryland Department of Assessment and Taxation, and MRIS;', 
    'Spoke with realtors engaged in the marketplace within the subject area as to the sale, leasing, and marketing times for similar type properties;',
    'Reviewed data of similar property sales in both MRIS and CoStar to verify the competitive environment of properties that are offered for sale that have similar characteristics as the subject property;',
    f'Reviewed material available from the {subject_county} Planning and Zoning Department regarding the subject property zoning and surrounding area;',
    'Reported results using USPAP prescribed content for a summary appraisal report;',
    'It is important to note that the opinion of value provided in this report is based on the assumption that the property is not negatively affected by the existence of hazardous substances or other environmental conditions.',
    'Real Estate Appraisers are not experts in the identification of hazardous substances or other environmental conditions. During the property inspection and inquiries of the subject site, the appraisers did not become aware of any information that would lead to the conclusion that any negative environmental conditions exist on the subject property.',
    'There are no non-realty sources of income such as business income, income attributable to intangibles, and priced premiums associated with non-market or subsidized financing in accordance with the proper identification of the property rights appraised.'
]

# Add each item as a bulleted list item
for item in items:
    paragraph = doc.add_paragraph(item, style='Custom Bullet Style')
    run = paragraph.runs[0]
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)

    # Ensure the font is applied correctly (Word-specific setting)
    r = run._element
    rPr = r.find(qn('w:rPr'))
    rFonts = OxmlElement('w:rFonts')
    rFonts.set(qn('w:ascii'), 'Times New Roman')
    rFonts.set(qn('w:hAnsi'), 'Times New Roman')
    rPr.append(rFonts)
    
# Add a page break
doc.add_page_break()
    


    
    
    
#### Pages 8 and 9






paragraph = doc.add_paragraph()
run = paragraph.add_run("ASSUMPTIONS & LIMITING CONDITIONS")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")  

# List of items for the bulleted list
items = appraisal_statements = [
    "Unless otherwise stated, the value appearing in this appraisal represents the opinion of the Market Value or the Value Defined as of the date specified. Market Value of real estate is affected by national and local economic conditions and, consequently, will vary with future changes in such conditions.",
    "The value estimated in this appraisal report is gross, without consideration given to any encumbrance, restriction or question of title, unless specifically defined.",
    "The value is reported in dollars on the basis of the currency prevailing at the date of appraisal.",
    "This appraisal report covers only the property described and any values or rates utilized are not to be construed as applicable to any other property, however similar the properties might be.",
    "It is assumed that the title to the premises is good, that the legal description is correct, that the improvements are entirely and correctly located on the property described, and that there are no encroachments on this property. However, no investigation or survey has been made.",
    "This appraisal expresses my opinion, and the employment to make this appraisal was in no way contingent upon the reporting or predetermined value or conclusion.",
    "No responsibility is assumed for matters legal in nature, nor is any opinion of title rendered. In the performance of my investigation and analysis leading to the conclusions reached herein, the statements of others were relied on.",
    "Neither all nor any part of the contents of this report (especially any conclusions, the identity of the appraiser or the firm with which he is connected, or any reference to the Appraisal Institute or any of its designations) shall be disseminated to the public through advertising media, public relations media, news media, sales media or any other public means of communication without my prior written consent and approval.",
    "It is assumed that there are no hidden or unapparent conditions of the property, subsoil, or structures which would render it more or less valuable. The Appraiser assumes no responsibility for such conditions or the engineering which might be required to discover these factors.",
    "Unless otherwise stated in this report, the existence of hazardous substances, including without limitation asbestos, polychlorinated biphenyls, petroleum leakage, or agricultural chemicals, which may or may not be present on the property, or other environmental conditions, were not called to the attention of the appraiser, nor did the appraiser become aware of such during the appraiser's inspection. The appraiser has no knowledge of the existence of such materials on or in the property unless otherwise stated. The appraiser, however, is not qualified to test for such substances or conditions. If the presence of such substances, such as asbestos, formaldehyde foam insulation, or other hazardous substances or environmental conditions, may affect the value of the property, the value estimated is predicated on the assumption that there is no such proximity thereto that would cause a loss in value. No responsibility is assumed for any such conditions, nor for any expertise or engineering knowledge required to discover them.",
    "All mortgages, liens, encumbrances, leases, and servitudes have been disregarded unless so specified within the report. The property is appraised as though under responsible ownership and competent management.",
    "It is assumed that there is full compliance with all applicable federal, state and local environmental regulations and laws unless non-compliance is stated, defined and considered in the appraisal report.",
    "It is assumed that all required licenses, consents or other legislative or administrative authority from any local, state or national government or private entity or organization have been or can be obtained or renewed for any use on which the value estimate contained in this report is based. It is assumed that a permit for the construction of the entire entity is in place.",
    "It is assumed that the subject property and the market data used within this report does not have a level of Radon in excess of the limits established by the Environmental Protection Agency (EPA).",
    "The appraiser will not be required to give testimony or appear in court because of having made this appraisal unless previous arrangements are made with the appraiser prior to the writing of the report.",
    "Assumed that live and dead loads within the structure do not exceed the design feature of the building.",
    "Assumed that the soils on the subject site will support the building improvements currently located on the site.",
    "Assumed that subject property owners have all appropriate permits and licenses required to operate the existing business. In addition, I assume that the subject property owners have the requisite management expertise for the continued operation of their business.",
    "Pertaining to the Americans with Disabilities Act (ADA), we have not made a specific compliance survey and analysis of this property to determine whether or not it is in conformity with the various detailed requirements of the ADA. It is possible that a compliance survey of the property together with a detailed analysis of the requirements of the ADA could reveal that the property is not in compliance with one or more of the requirements of the act. If so, this fact could have a negative effect upon the value of the property. Since we have no direct evidence relating to this issue, we did not consider possible noncompliance with the requirements of ADA in estimating the value of the property.",
    "The scope of this appraisal does not include personal property.",
    "This appraisal report was completed without the benefit of a title report. It is assumed that the title to the premises is good, that the legal description is correct, that the improvements are entirely and correctly located on the property described, and that there are no encroachments on this property other than what is noted in the report.",
    f"The property tax map and information from the {subject_county} land records were used as reference to determine/verify the site size and dimensions and it was assumed to be an accurate representation of the subject site. A formal survey was not completed as part of this assignment."
]


# Add each item as a bulleted list item
for item in appraisal_statements:
    paragraph = doc.add_paragraph(item, style='Custom Bullet Style')
    run = paragraph.runs[0]
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)

    # Ensure the font is applied correctly (Word-specific setting)
    r = run._element
    rPr = r.find(qn('w:rPr'))
    rFonts = OxmlElement('w:rFonts')
    rFonts.set(qn('w:ascii'), 'Times New Roman')
    rFonts.set(qn('w:hAnsi'), 'Times New Roman')
    rPr.append(rFonts)
    
# Add a page break
doc.add_page_break()







#### Page 10








composer = Composer(doc)

if interest_appraised == "Fee Simple" and valuation_choice == "Current Market Value 'As-Is'":
    key_doc = Document('fee_simple_current.docx')
elif interest_appraised == 'Fee Simple' and valuation_choice == "Retrospective Market Value":
    key_doc = Document('fee_simple_retrospective.docx')
elif interest_appraised == 'Leased Fee' and valuation_choice == "Current Market Value 'As-Is'":
    key_doc = Document('leased_fee_current.docx')
else:
    key_doc = Document('leased_fee_retrospective.docx')

# Append the second document
composer.append(key_doc)

# Add a page break
doc.add_page_break()








#### Page 11







# Add a table with 9 rows and 2 columns
table = doc.add_table(rows=9, cols=2)

# Set the table to not autofit
table.autofit = False

# Set the preferred width for the entire table
table.width = Inches(6.68)  # Adjust this value as needed

# Set the width for each column
column_widths = [Inches(1.56), Inches(5.12)]  # Adjust these values as needed
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

exposure_time = "Exposure Time"
exposure_time_def = "The estimated length of time the property interest being appraised would have been offered on the market prior to the hypothetical consummation of a sale at market value on the effective date of the appraisal; a retrospective estimate based on an analysis of past events assuming a competitive and open market."
marketing_time = "Marketing Time"
marketing_time_def = "The time it takes an interest in real property to sell on the market subsequent to the date of appraisal."

left_col_vals = [
    "",
    "EXPOSURE AND MARKETING TIME",
    "",
    "",
    "",
    "",
    "",
    "",
    ""
]

right_col_vals = [
    "Exposure time as used in this appraisal report is defined as:",
    f"{exposure_time} {exposure_time_def}",
    "Based upon the market data from the sales of comparable properties in the market my conclusion of exposure time follows:",
    "Conclusion:",
    "Exposure Time              [ENTER TIME HERE]",
    f"{marketing_time} {marketing_time_def}",
    "Based upon the market data from the sales of comparable properties in the market my conclusion of exposure time follows:",
    "Conclusion:",
    "Marketing Time             [ENTER TIME HERE]"
]

for i in range(9):
    # Left column with values made bold
    cell_left = table.cell(i, 0)
    paragraph_left = cell_left.paragraphs[0]
    add_bold_text(paragraph_left, left_col_vals[i])

    # Right column with values from the provided list
    cell_right = table.cell(i, 1)
    paragraph_right = cell_right.paragraphs[0]
    if i == 1:  
        add_bold_text(paragraph_right, exposure_time)     
        add_italic_text(paragraph_right, f" {exposure_time_def}")  
    elif i == 5:
        add_bold_text(paragraph_right, marketing_time)     
        add_italic_text(paragraph_right, f" {marketing_time_def}")  
    else:
        paragraph_right.add_run(right_col_vals[i])

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        for run in paragraph.runs:
            run.font.name = 'Times New Roman'
            run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(6)
        paragraph.paragraph_format.space_after = Pt(0)

# Add a page break
doc.add_page_break()






#### Page 12







# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("IDENTIFICATION OF THE SUBJECT PROPERTY")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Add a table with 6 rows and 2 columns
table = doc.add_table(rows=6, cols=2)

# Set the table to not autofit
table.autofit = False

# Set the preferred width for the entire table
table.width = Inches(6.68)

# Set the width for each column
column_widths = [Inches(1.56), Inches(5.12)]
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

if vac_or_imp == 'vacant':
    left_col_vals = [
    "PROPERTY ADDRESS",
    "LEGAL DESCRIPTION",
    "SUBJECT PROPERTY SUMMARY",
    "",
    "TAX MAP",
    ""
    ]
else:
    left_col_vals = [
    "PROPERTY ADDRESS",
    "LEGAL DESCRIPTION",
    "SUBJECT PROPERTY SUMMARY",
    "",
    "",
    "TAX MAP"
    ]

if vac_or_imp == "vacant" and parcel_shape == "rectangular":
    right_col_vals = [
    f"{property_address}",
    f"Tax Account: {tax_account}\n{lot_size} {lot_unit} of land\nLiber {liber}/Folio {folio}\nMap {map_num}/Grid {grid}/Parcel {parcel}\n{election_district} Election District\nCensus Tract: {census_tract}\n{city} {state} {zip_code}",  # Leave this blank; we will add legal description paragraphs separately
    f"The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}.",
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}",
    "",
    ""
    ]
elif vac_or_imp == "vacant" and parcel_shape == "irregular":
    right_col_vals = [
    f"{property_address}",
    f"Tax Account: {tax_account}\n{lot_size} {lot_unit} of land\nLiber {liber}/Folio {folio}\nMap {map_num}/Grid {grid}/Parcel {parcel}\n{election_district} Election District\nCensus Tract: {census_tract}\n{city} {state} {zip_code}",  # Leave this blank; we will add legal description paragraphs separately
    f"The subject property is located on an {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}.",
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}",
    "",
    ""
    ]
elif vac_or_imp == "improved" and parcel_shape == "rectangular":
    right_col_vals = [
    f"{property_address}",
    f"Tax Account: {tax_account}\n{lot_size} {lot_unit} of land\nLiber {liber}/Folio {folio}\nMap {map_num}/Grid {grid}/Parcel {parcel}\n{election_district} Election District\nCensus Tract: {census_tract}\n{city} {state} {zip_code}",  # Leave this blank; we will add legal description paragraphs separately
    f"The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}.",
    f"{property_paragraph}",
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}",
    ""
    ]
else:
    right_col_vals = [
    f"{property_address}",
    f"Tax Account: {tax_account}\n{lot_size} {lot_unit} of land\nLiber {liber}/Folio {folio}\nMap {map_num}/Grid {grid}/Parcel {parcel}\n{election_district} Election District\nCensus Tract: {census_tract}\n{city} {state} {zip_code}",  # Leave this blank; we will add legal description paragraphs separately
    f"The subject property is located on an {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}.",
    f"{property_paragraph}",
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}",
    ""
    ]

for i in range(6):
    # Left column with values made bold
    cell_left = table.cell(i, 0)
    paragraph_left = cell_left.paragraphs[0]
    add_bold_text(paragraph_left, left_col_vals[i])

    # Right column with values from the provided list
    cell_right = table.cell(i, 1)
    paragraph_right = cell_right.paragraphs[0]
    
    # Add text from right_col_vals
    paragraph_right.add_run(right_col_vals[i])

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        for run in paragraph.runs:
            run.font.name = 'Times New Roman'
            run.font.size = Pt(12)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(6)
        paragraph.paragraph_format.space_after = Pt(0)
        
# Add a page break
doc.add_page_break()








#### Page 13








# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("SUBJECT PROPERTY HISTORY")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property was transferred from {grantor} ('grantor') to {grantee} ('grantee') on {transfer_date}. This {terms} transaction was recorded in Liber {liber} Folio {folio} of the {subject_county} land records.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0) 
prep_paragraph.paragraph_format.line_spacing = 1.5

# Add a page break
doc.add_page_break()







#### Page 13








composer = Composer(doc)

if subject_county == "Frederick County":
    region_doc = Document('frederick_regional.docx')
    composer.append(region_doc)
elif subject_county == "Washington County":
    region_doc = Document('washington_regional.docx')
    composer.append(region_doc)
    doc.add_page_break()
else:
    region_doc = Document('allegany_regional.docx')
    composer.append(region_doc)










####Page after regional data









# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("NEIGHBORHOOD DATA")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")
paragraph.paragraph_format.space_after = Pt(6) 

doc.add_paragraph()
doc.add_paragraph()
doc.add_paragraph()
doc.add_paragraph()

# Add a table with 9 rows and 2 columns
table = doc.add_table(rows=6, cols=2)
table.cell(0, 0).merge(table.cell(0, 1))
table.cell(0, 0).paragraphs[0].alignment = 1
table.autofit = False
table.alignment = WD_ALIGN_PARAGRAPH.CENTER
# table.width = Inches(4.5)  # Adjust this value as needed

# Set the width for each column
column_widths = [Inches(3.44), Inches(1.06)]  # Adjust these values as needed
for i, width in enumerate(column_widths):
    for cell in table.columns[i].cells:
        cell.width = width

left_col_vals = [
    f"Census Tract: {census_tract} Summary",
    "2023 Estimated Median Family Income",
    "Tract Population",
    "Number of Households",
    "1-4 Family Units",
    "Owner-Occupied Units"
]

right_col_vals = [
    "",
    "",
    "",
    "",
    "",
    ""
]

for i in range(6):
    # Left column with values
    cell_left = table.cell(i, 0)
    paragraph_left = cell_left.paragraphs[0]
    if i == 0:
        run_left = paragraph_left.add_run(left_col_vals[i])
        run_left.bold = True  # Make the text bold
    else:
        paragraph_left.add_run(left_col_vals[i])
    cell_right = table.cell(i, 1)
    paragraph_right = cell_right.paragraphs[0]
    paragraph_right.add_run(right_col_vals[i])

# Set font style and size for all cells
for row in table.rows:
    for cell in row.cells:
        paragraph = cell.paragraphs[0]
        for run in paragraph.runs:
            run.font.name = 'Calibri'
            run.font.size = Pt(11)

        # Adjust row spacing
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)

# Set borders for all cells
for row in table.rows:
    for cell in row.cells:
        set_cell_borders(cell, top_color="000000", bottom_color="000000", left_color="000000", right_color="000000", border_width=4)

# Set the fill color for the top row
for cell in table.row_cells(0):  # top row (index 0)
    set_cell_fill(cell, "A6A6A6")

doc.add_paragraph()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("The following is a census tract map with the subject property indicated using a red dot:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0)   

# Add a page break
doc.add_page_break()






#### Real Estate Tax Analysis







# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("REAL ESTATE TAX ANALYSIS")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is located within {subject_county}, Maryland. Article 15 of the Declaration of Rights of Maryland’s Constitution requires that all real property is assessed and taxed uniformly. State law specifically requires that all taxable property shall be assessed based on its fair market value. The Maryland Department of Assessments and Taxation is responsible for appraising all real property and business personal property within the entire State of Maryland. Properties are appraised once every three years and property owners are notified of any change in their assessed value. Assessments are certified by the Department to local governments where they are converted into property tax bill by applying the appropriate property tax rates. Property tax rates vary widely. No restrictions or limitations on property taxes are imposed by the State, enabling cities and counties to set tax rates at the level required to fund governmental services. These rates can increase, decrease, or remain the same from year to year. If the proposed tax rate increases the total property tax revenues, the governing body must advertise that fact and hold a public hearing on the new tax rate. This is called the Constant Tax Yield Rate process.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0) 
prep_paragraph.paragraph_format.line_spacing = 1.5

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("The overall tax rate is a combination of State, county, and in some cases municipal tax rates. Property taxes are expressed as a dollar amount per $100 of assessed value. The following chart reflects the assessed value for real estate tax purposes with the effective date of [ENTER 'AS OF' DATE HERE]:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0) 
prep_paragraph.paragraph_format.line_spacing = 1.5

paragraph = doc.add_paragraph()
run = paragraph.add_run(f"Tax Account: {tax_account}")
run.bold = True
run.font.name = 'Calibri'
run.font.size = Pt(11)
paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")
paragraph.paragraph_format.space_before = Pt(6)
paragraph.paragraph_format.space_after = Pt(6)

doc.add_paragraph()

if subject_county == "Frederick County":
    prep_paragraph = doc.add_paragraph()
    prep_run = prep_paragraph.add_run("The property is subject to taxes levied by the State of Maryland and Frederick County. Frederick County taxes include an Urban Fire/Rescue tax and System Benefit Charge which is administered by the Frederick County Waste Management Division. Frederick County taxes include a System Benefit Charge which is administered by the Frederick County Waste Management Division. The purpose of the charge is to pay a portion of the reasonably anticipated capital costs and operating costs for the disposal of solid waste. The charge is applied countywide, including within the municipalities. All information for the assessment of a property is based on the Maryland State Department of Assessments and Taxation’s (SDAT) records. Non-residential rates are applied to properties with improvement values greater than $5,000 and the number of billing units is based on several factors including the Bureau of Public Roads Use Codes (BPRUC) that establishes the primary use of the property, enclosed structure area and the land use code. The land use code will determine the level of waste generated by each property. The following identifies the tax charges for the subject property:")
    prep_run.bold = False
    prep_run.font.size = Pt(12)
    prep_run.font.name = 'Times New Roman'
    prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
    prep_paragraph.paragraph_format.space_before = Pt(6)
    prep_paragraph.paragraph_format.space_after = Pt(0) 
    prep_paragraph.paragraph_format.line_spacing = 1.5

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Tax Account: {tax_account}")
prep_run.bold = True
prep_run.underline = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0) 
prep_paragraph.paragraph_format.line_spacing = 1.5

doc.add_paragraph()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("As of the date of the report date, there is a zero balance for this account.")
prep_run.bold = False
prep_run.underline = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(0) 
prep_paragraph.paragraph_format.line_spacing = 1.5

# Add a page break
doc.add_page_break()







#### Market Analysis Section







composer = Composer(doc)

market_doc = Document('market_analysis.docx')

# Append the second document
composer.append(market_doc)

# Add a page break
doc.add_page_break()








#### Zoning








# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("ZONING")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")
       
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc} Below is a zoning map of the subject property and surrounding area from {subject_county} Property Explorer:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)
prep_paragraph.paragraph_format.line_spacing = 1.5

# Add a page break
doc.add_page_break()









#### Site Data










# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("SITE DATA")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"""The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}. A satellite image of the subject site is reflected below:""")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)
prep_paragraph.paragraph_format.line_spacing = 1.5

# Add a page break
doc.add_page_break()

composer = Composer(doc)

site_doc = Document('site_data.docx')

# Capitalize the first letter of parcel_shape while leaving the rest unchanged
formatted_parcel_shape = parcel_shape[0].upper() + parcel_shape[1:]

# First cell: clear content, add new content, and format
cell_1 = site_doc.tables[0].cell(6, 1)
cell_1.text = ""
run_1 = cell_1.paragraphs[0].add_run(f"{road_frontage} feet of frontage along {street_name}.")
run_1.font.size = Pt(10)
run_1.font.name = 'Times New Roman'
cell_1.paragraphs[0].paragraph_format.space_before = Pt(6)

# Second cell: clear content, add new content, and format
cell_2 = site_doc.tables[0].cell(8, 1)
cell_2.text = ""
run_2 = cell_2.paragraphs[0].add_run(f"{lot_size_formatted} {lot_unit}.")
run_2.font.size = Pt(10)
run_2.font.name = 'Times New Roman'
cell_2.paragraphs[0].paragraph_format.space_before = Pt(6)

# Third cell: clear content, add new content, and format
cell_3 = site_doc.tables[0].cell(9, 1)
cell_3.text = ""
run_3 = cell_3.paragraphs[0].add_run(f"{formatted_parcel_shape}.")
run_3.font.size = Pt(10)
run_3.font.name = 'Times New Roman'
cell_3.paragraphs[0].paragraph_format.space_before = Pt(6)


composer.append(site_doc)

# Add a page break
doc.add_page_break()

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("FEMA FLOOD MAP:  ; Effective on ")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_run.font.underline = True 
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("It is important to note that the opinion of value provided in this report assumes that the property is not negatively affected by the existence of hazardous substances or other environmental conditions. Real Estate Appraisers are not experts in the identification of hazardous substances or other environmental conditions. During the property inspection and inquiries of the subject site, the appraisers did not become aware of any information that would lead to the conclusion that any negative environmental conditions exist on the subject property. The presence of contamination could negatively impact our opinion of value.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)
prep_paragraph.paragraph_format.line_spacing = 1.5

# Add a page break
doc.add_page_break()








#### Description of the improvements







if vac_or_imp != 'vacant':
    # Adding a paragraph with specific formatting
    paragraph = doc.add_paragraph()
    run = paragraph.add_run("DESCRIPTION OF THE IMPROVEMENTS")
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(14)
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

    prep_paragraph = doc.add_paragraph()
    prep_run = prep_paragraph.add_run(f"{property_paragraph} The following table summarizes each of the major building elements:")
    prep_run.bold = False
    prep_run.font.size = Pt(12)
    prep_run.font.name = 'Times New Roman'
    prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
    prep_paragraph.paragraph_format.space_before = Pt(6)
    prep_paragraph.paragraph_format.space_after = Pt(6)
    prep_paragraph.paragraph_format.line_spacing = 1.5

    composer = Composer(doc)

    building_element_doc = Document('building_element_summary.docx')

    # First cell: clear content, add new content, and format
    cell_1 = building_element_doc.tables[0].cell(1, 1)
    cell_1.text = ""
    run_1 = cell_1.paragraphs[0].add_run(f"{year_built}.")
    run_1.font.size = Pt(10)
    run_1.font.name = 'Times New Roman'

    # First cell: clear content, add new content, and format
    cell_1 = building_element_doc.tables[0].cell(2, 1)
    cell_1.text = ""
    run_1 = cell_1.paragraphs[0].add_run(f"{property_type}.")
    run_1.font.size = Pt(10)
    run_1.font.name = 'Times New Roman'
    
    # First cell: clear content, add new content, and format
    cell_1 = building_element_doc.tables[0].cell(6, 1)
    cell_1.text = ""
    run_1 = cell_1.paragraphs[0].add_run(f"{building_sf_formatted}.")
    run_1.font.size = Pt(10)
    run_1.font.name = 'Times New Roman'

    # Append the second document
    composer.append(building_element_doc)

    # Add a page break
    doc.add_page_break()






#### Subject Property Photos






# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("SUBJECT PROPERTY PHOTOS")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Add a page break
doc.add_page_break()







#### Highest and Best Use






composer = Composer(doc)

highest_and_best_use_doc = Document('highest_and_best_use.docx')

#### As Vacant Table

# First cell: clear content, add new content, and format
cell_1 = highest_and_best_use_doc.tables[0].cell(0, 1)

# Clear existing text
cell_1.text = ""  # Clear the content

# Add text directly to the cell
cell_1.text = f"The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}."
# Set font size and name
cell_1.paragraphs[0].runs[0].font.size = Pt(12)
cell_1.paragraphs[0].runs[0].font.name = 'Times New Roman'
# Add spacing after the paragraph
cell_1.paragraphs[0].paragraph_format.space_after = Pt(6)

# Second cell: clear content, add new content, and format
cell_2 = highest_and_best_use_doc.tables[0].cell(1, 1)

# Clear existing text
cell_2.text = ""  # Clear the content

# Add text directly to the cell
cell_2.text = f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}"
# Set font size and name
cell_2.paragraphs[0].runs[0].font.size = Pt(12)
cell_2.paragraphs[0].runs[0].font.name = 'Times New Roman'
# Add spacing after the paragraph
cell_2.paragraphs[0].paragraph_format.space_after = Pt(6)

#### As Improved Table

# First cell: clear content, add new content, and format
cell_1 = highest_and_best_use_doc.tables[1].cell(0, 1)

# Clear existing text
cell_1.text = ""  # Clear the content

# Add text directly to the cell
cell_1.text = f"The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size_formatted} {lot_unit}, according to the land records of {subject_county}. This site features approximately {road_frontage} feet of frontage along {street_name}."
# Set font size and name
cell_1.paragraphs[0].runs[0].font.size = Pt(12)
cell_1.paragraphs[0].runs[0].font.name = 'Times New Roman'
# Add spacing after the paragraph
cell_1.paragraphs[0].paragraph_format.space_after = Pt(6)

# Second cell: clear content, add new content, and format
cell_2 = highest_and_best_use_doc.tables[1].cell(1, 1)

# Clear existing text
cell_2.text = ""  # Clear the content

# Add text directly to the cell
cell_2.text = f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}"
# Set font size and name
cell_2.paragraphs[0].runs[0].font.size = Pt(12)
cell_2.paragraphs[0].runs[0].font.name = 'Times New Roman'
# Add spacing after the paragraph
cell_2.paragraphs[0].paragraph_format.space_after = Pt(6)

# Append the second document
composer.append(highest_and_best_use_doc)


# # Save the document
doc.save("appraisal_report.docx")

Enter the client's name:  Ms. Pamela Phillips Valois
Enter the client's mailing address (Street Address, County/City, State Zip):  47 East All Saints Street, Frederick, Maryland 21701
Enter the property address  (Street Address, County/City, State Zip):  5815B Jefferson Pike, Frederick, Maryland 21703


Select the county of the subject property:
1: Frederick County
2: Washington County
3: Allegany County


Enter the number corresponding to county of the subject property:  1



Select the zoning of the subject property:
1: Agricultural
2: Low-Density Residential
3: Low-Density Residential
4: Medium-Density Residential
5: Medium-Density Residential
6: High-Density Residential
7: High-Density Residential
8: Village Center
9: General Commercial
10: Mixed Use
11: Limited Industrial
12: General Industrial
13: Mineral Mining
14: Office/Research Industrial


Enter the number corresponding to the zoning of the subject property:  1
Enter the report date:  TBD
Enter the PRG reference number:  2011-241106-Farm
Enter the tax account:  23-437597


...

Enter the number of your choice (1 or 2):  2


Is this appraisal for an estate?
1. Yes
2. No


Enter the number of your choice (1 or 2):  1
Please enter the name of the decedent:  Mr. William J. Valois, Jr.
Please enter the date of death (e.g., MM/DD/YYYY):  December 8, 2023
Please enter the estate number:  Unknown
Please enter the date of inspection (e.g., MM/DD/YYYY):  TBD


What is the interest appraised?
1. Fee Simple.
2. Leased Fee.


Please enter the number corresponding to your choice:  1
Prepared by:  John S. Molesworth
